In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_wudao_zhaiwu=pd.DataFrame(columns=columns)
channel_wudao_zhaiwu

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').click()  #选择“舞蹈”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'舞蹈'

In [8]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择“宅舞”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'宅舞'

In [9]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [10]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_wudao_zhaiwu.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 19:34:18.729561


<ipython-input-10-a6efb9cdb962>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-a6efb9cdb962>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-a6efb9cdb962>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1KJ411C7SB视频属性的抓取
BV号为1KJ411C7SB的视频属性抓取结束


<ipython-input-10-a6efb9cdb962>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1R7411k7Ef视频属性的抓取
BV号为1R7411k7Ef的视频属性抓取结束
开始BV号为1CJ411V7DL视频属性的抓取
BV号为1CJ411V7DL的视频属性抓取结束
开始BV号为1Y7411i7YA视频属性的抓取
BV号为1Y7411i7YA的视频属性抓取结束
开始BV号为1SJ41157qN视频属性的抓取
BV号为1SJ41157qN的视频属性抓取结束
开始BV号为12J411J7L7视频属性的抓取
BV号为12J411J7L7的视频属性抓取结束
开始BV号为18J411j7ef视频属性的抓取
BV号为18J411j7ef的视频属性抓取结束
开始BV号为1J7411q78g视频属性的抓取
BV号为1J7411q78g的视频属性抓取结束
开始BV号为1eJ411H7d2视频属性的抓取
BV号为1eJ411H7d2的视频属性抓取结束
开始BV号为1A7411e7DW视频属性的抓取
BV号为1A7411e7DW的视频属性抓取结束
开始BV号为1F7411Y7qs视频属性的抓取
BV号为1F7411Y7qs的视频属性抓取结束
开始BV号为1w7411i74f视频属性的抓取
BV号为1w7411i74f的视频属性抓取结束
开始BV号为1VJ411L72s视频属性的抓取
BV号为1VJ411L72s的视频属性抓取结束
开始BV号为1yJ411h7ZH视频属性的抓取
BV号为1yJ411h7ZH的视频属性抓取结束
开始BV号为1XJ411J7vE视频属性的抓取
BV号为1XJ411J7vE的视频属性抓取结束
开始BV号为12J411H7oK视频属性的抓取
BV号为12J411H7oK的视频属性抓取结束
开始BV号为1s7411v77F视频属性的抓取
BV号为1s7411v77F的视频属性抓取结束
开始BV号为13J411p7tJ视频属性的抓取
BV号为13J411p7tJ的视频属性抓取结束
开始BV号为1a741167PL视频属性的抓取
BV号为1a741167PL的视频属性抓取结束
开始BV号为11J411b7i7视频属性的抓取
BV号为11J411b7i7的视频属性抓取结束
开始BV号为1d7411i7ma视频属性的抓取
BV号为1d7411i7ma的视频属性抓取结束
开始BV号为1x7411i7Af视频属性的抓取
BV号为1x7411i7Af的视

...

<ipython-input-10-a6efb9cdb962>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Q5411s7AT视频属性的抓取
BV号为1Q5411s7AT的视频属性抓取结束
开始BV号为1BV411C7hr视频属性的抓取
BV号为1BV411C7hr的视频属性抓取结束
开始BV号为1aK411W794视频属性的抓取
BV号为1aK411W794的视频属性抓取结束
开始BV号为1rp4y197mg视频属性的抓取
BV号为1rp4y197mg的视频属性抓取结束
开始BV号为1jA411q78C视频属性的抓取
BV号为1jA411q78C的视频属性抓取结束
开始BV号为14k4y167Xr视频属性的抓取
BV号为14k4y167Xr的视频属性抓取结束
开始BV号为1Ki4y1b78J视频属性的抓取
BV号为1Ki4y1b78J的视频属性抓取结束
开始BV号为1te411p7Ah视频属性的抓取
BV号为1te411p7Ah的视频属性抓取结束
开始BV号为1F541147gu视频属性的抓取
BV号为1F541147gu的视频属性抓取结束
开始BV号为1sp4y1971z视频属性的抓取
BV号为1sp4y1971z的视频属性抓取结束
开始BV号为1WK4y1t71B视频属性的抓取
BV号为1WK4y1t71B的视频属性抓取结束
开始BV号为1SV411k7iG视频属性的抓取
BV号为1SV411k7iG的视频属性抓取结束
开始BV号为12p4y1978y视频属性的抓取
BV号为12p4y1978y的视频属性抓取结束
开始BV号为1W54y1X7oW视频属性的抓取
BV号为1W54y1X7oW的视频属性抓取结束
开始BV号为1qk4y1k7N7视频属性的抓取
BV号为1qk4y1k7N7的视频属性抓取结束
开始BV号为18V411C7ja视频属性的抓取
BV号为18V411C7ja的视频属性抓取结束
开始BV号为1jZ4y1p7pj视频属性的抓取
BV号为1jZ4y1p7pj的视频属性抓取结束
开始BV号为15t4y1m7Du视频属性的抓取
BV号为15t4y1m7Du的视频属性抓取结束
开始BV号为1oV411C7Wx视频属性的抓取
BV号为1oV411C7Wx的视频属性抓取结束
开始BV号为1if4y1m7hU视频属性的抓取
BV号为1if4y1m7hU的视频属性抓取结束
开始BV号为1QZ4y1p7ei视频属性的抓取
BV号为1QZ4y1p7ei的视

BV号为1Nt4y1D7Q4的视频属性抓取结束
开始BV号为1cZ4y1g71Q视频属性的抓取
BV号为1cZ4y1g71Q的视频属性抓取结束
开始BV号为12p4y1i7H5视频属性的抓取
BV号为12p4y1i7H5的视频属性抓取结束
开始BV号为1Ra4y1h7zS视频属性的抓取
BV号为1Ra4y1h7zS的视频属性抓取结束
开始BV号为1at4y1X7Bp视频属性的抓取
BV号为1at4y1X7Bp的视频属性抓取结束
开始BV号为1454y1q7Ea视频属性的抓取
BV号为1454y1q7Ea的视频属性抓取结束
开始BV号为115411Y757视频属性的抓取
BV号为115411Y757的视频属性抓取结束
开始BV号为1Ki4y1V7Eb视频属性的抓取
BV号为1Ki4y1V7Eb的视频属性抓取结束
开始BV号为1Yz4y1D7B8视频属性的抓取
BV号为1Yz4y1D7B8的视频属性抓取结束
开始BV号为1qi4y1G7JX视频属性的抓取
BV号为1qi4y1G7JX的视频属性抓取结束
开始BV号为15V411r776视频属性的抓取
BV号为15V411r776的视频属性抓取结束
开始BV号为1it4y1Q7bE视频属性的抓取
BV号为1it4y1Q7bE的视频属性抓取结束
开始BV号为1Cz4y1X75p视频属性的抓取
BV号为1Cz4y1X75p的视频属性抓取结束
开始BV号为1xp4y1q7QZ视频属性的抓取
BV号为1xp4y1q7QZ的视频属性抓取结束
开始BV号为1zi4y137ta视频属性的抓取
BV号为1zi4y137ta的视频属性抓取结束
开始BV号为1xh411o7GK视频属性的抓取
BV号为1xh411o7GK的视频属性抓取结束
开始BV号为1aD4y1Q7aY视频属性的抓取
BV号为1aD4y1Q7aY的视频属性抓取结束
开始BV号为1T5411Y7sT视频属性的抓取
BV号为1T5411Y7sT的视频属性抓取结束
开始BV号为1nt4y1Q7jk视频属性的抓取
BV号为1nt4y1Q7jk的视频属性抓取结束
开始BV号为1ZD4y1m7Pf视频属性的抓取
BV号为1ZD4y1m7Pf的视频属性抓取结束
开始BV号为1zz4y1D7R8视频属性的抓取
BV号为1zz4y1D7R8的视频属性抓取结束
开始BV号为1o54y1U7S8

BV号为1rK4y1Y7so的视频属性抓取结束
开始BV号为1eA411Y7LA视频属性的抓取
BV号为1eA411Y7LA的视频属性抓取结束
开始BV号为1Sk4y1177V视频属性的抓取
BV号为1Sk4y1177V的视频属性抓取结束
开始BV号为11T4y1j7GP视频属性的抓取
BV号为11T4y1j7GP的视频属性抓取结束
开始BV号为1j54y1U7cR视频属性的抓取
BV号为1j54y1U7cR的视频属性抓取结束
开始BV号为1HZ4y1M7Kr视频属性的抓取
BV号为1HZ4y1M7Kr的视频属性抓取结束
开始BV号为1mD4y1U7h4视频属性的抓取
BV号为1mD4y1U7h4的视频属性抓取结束
开始BV号为12Z4y1M7yP视频属性的抓取
BV号为12Z4y1M7yP的视频属性抓取结束
开始BV号为1my4y1C7ZJ视频属性的抓取
BV号为1my4y1C7ZJ的视频属性抓取结束
开始BV号为16h41197Vt视频属性的抓取
BV号为16h41197Vt的视频属性抓取结束
开始BV号为18C4y1t7zp视频属性的抓取
BV号为18C4y1t7zp的视频属性抓取结束
开始BV号为1sZ4y1N78q视频属性的抓取
BV号为1sZ4y1N78q的视频属性抓取结束
开始BV号为1sA411J7vJ视频属性的抓取
BV号为1sA411J7vJ的视频属性抓取结束
开始BV号为1ak4y1y7UN视频属性的抓取
BV号为1ak4y1y7UN的视频属性抓取结束
开始BV号为1Ft4y1q7WS视频属性的抓取
BV号为1Ft4y1q7WS的视频属性抓取结束
开始BV号为1qA411E7P1视频属性的抓取
BV号为1qA411E7P1的视频属性抓取结束
开始BV号为1A54y117fd视频属性的抓取
BV号为1A54y117fd的视频属性抓取结束
开始BV号为1Pt4y1q7aH视频属性的抓取
BV号为1Pt4y1q7aH的视频属性抓取结束
开始BV号为1z54y1C7gz视频属性的抓取
BV号为1z54y1C7gz的视频属性抓取结束
开始BV号为1LD4y1m7VJ视频属性的抓取
BV号为1LD4y1m7VJ的视频属性抓取结束
开始BV号为1df4y1S73T视频属性的抓取
BV号为1df4y1S73T的视频属性抓取结束
开始BV号为18a4y1E72S

BV号为1dy4y1r7Wz的视频属性抓取结束
开始BV号为1WA411E7DX视频属性的抓取
BV号为1WA411E7DX的视频属性抓取结束
开始BV号为1yK411A7fC视频属性的抓取
BV号为1yK411A7fC的视频属性抓取结束
开始BV号为1j54y1C7PZ视频属性的抓取
BV号为1j54y1C7PZ的视频属性抓取结束
开始BV号为1Wy4y1B7vd视频属性的抓取
BV号为1Wy4y1B7vd的视频属性抓取结束
开始BV号为17Z4y1V76U视频属性的抓取
BV号为17Z4y1V76U的视频属性抓取结束
开始BV号为1La4y1s7uJ视频属性的抓取
BV号为1La4y1s7uJ的视频属性抓取结束
开始BV号为1NT4y1F77F视频属性的抓取
BV号为1NT4y1F77F的视频属性抓取结束
开始BV号为15v411r73b视频属性的抓取
BV号为15v411r73b的视频属性抓取结束
开始BV号为1Fk4y1y7Z6视频属性的抓取
BV号为1Fk4y1y7Z6的视频属性抓取结束
开始BV号为1Ma4y1s7Gc视频属性的抓取
BV号为1Ma4y1s7Gc的视频属性抓取结束
开始BV号为12z4y1o7G8视频属性的抓取
BV号为12z4y1o7G8的视频属性抓取结束
开始BV号为1A54y1C791视频属性的抓取
BV号为1A54y1C791的视频属性抓取结束
开始BV号为19t4y1Y7zP视频属性的抓取
BV号为19t4y1Y7zP的视频属性抓取结束
开始BV号为1qD4y1R7yE视频属性的抓取
BV号为1qD4y1R7yE的视频属性抓取结束
开始BV号为1NA411E7dT视频属性的抓取
BV号为1NA411E7dT的视频属性抓取结束
开始BV号为19f4y1q75c视频属性的抓取
BV号为19f4y1q75c的视频属性抓取结束
开始BV号为17Z4y1V7et视频属性的抓取
BV号为17Z4y1V7et的视频属性抓取结束
开始BV号为1BD4y1d7H5视频属性的抓取
BV号为1BD4y1d7H5的视频属性抓取结束
开始BV号为1Pk4y1y7PS视频属性的抓取
BV号为1Pk4y1y7PS的视频属性抓取结束
开始BV号为1Cf4y1i7rp视频属性的抓取
BV号为1Cf4y1i7rp的视频属性抓取结束
开始BV号为1JD4y1d7P6

In [195]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-195-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_wudao_zhaiwu.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
6月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [13]:
channel_wudao_zhaiwu.to_json('channel_wudao_zhaiwu.json')